In [ ]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('Documents')[0],'Documents/BecaNLP/Utils'))

import NLPUtils as nlp
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Cargamos el dataset

In [ ]:
df = pd.read_csv('./train.csv',sep = '|')
df.columns = ['Pregunta', 'Intencion']
df = pd.read_csv('train.csv',sep = '|')
df['Intencion'] = df.Intencion.str.findall(r'\d+').apply(lambda x: int(x[0]))
categories = set(df['Intencion'].values)
name2idx = {name:idx for idx, name in enumerate(categories)}
idx2name = {idx:name for idx, name in enumerate(categories)}
_ = plt.hist(df['Intencion'].values,bins=max(df['Intencion']))
df.head()

# Clasificamos

Modelo baseline de clasificación

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

# Separamos en Train y Test:
X_train, X_test, y_train, y_test = train_test_split(df.Pregunta, df.Intencion, random_state = 0)

# Vectorizamos:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer(sublinear_tf=True)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

# Entrenamos:
clf = SVC(C=2)
clf.fit(X_train_tfidf, y_train)

# Predecimos:
y_train_pred = clf.predict(X_train_tfidf)
y_test_pred = clf.predict(X_test_tfidf)

# Evaluamos los resultados

In [ ]:
from NLPUtils.classification import get_score

metrics = ['accuracy','f1_macro', 'balanced_accuracy']
train_score = get_score(y_train,y_train_pred,metrics)
test_score = get_score(y_test,y_test_pred,metrics)
print('Resultados para train:')
print(train_score)
print('Resultados para test:')
print(test_score)